In [ ]:
!unzip /content/drive/MyDrive/superAI_lv2/hackathon/Final_5hacks/car_prediction/hackathon-online-data-science-and-big-data-ss-4.zip

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os

sp_sms = pd.read_csv('sample_submission.csv')
X_train = pd.read_csv('train_data.csv')
Y_train = pd.read_csv('train_target.csv')
df_test = pd.read_csv('test_data.csv')

In [ ]:
Y_train.head(), Y_train.Car.value_counts()

In [ ]:
df_test

In [ ]:
X_train.info(), df_test.info()

In [ ]:
ls_col_cat = list(X_train.select_dtypes(exclude=["number"]).columns)
ls_col_num = list(X_train._get_numeric_data().columns)

In [ ]:
len(ls_col_num)

# prepare

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# # Separate features and target variable from training data
X_num = X_train[ls_col_num]
X_num = X_num.drop(columns=['Id'])
X_cat = X_train[ls_col_cat]
y = Y_train['Car']

# For the test set, we only have features
X_test_num = df_test[ls_col_num]
X_test_num = X_test_num.drop(columns=['Id'])
X_test_cat = df_test[ls_col_cat]

In [ ]:
X_num.shape, X_cat.shape, X_test_num.shape, X_test_cat.shape

In [ ]:
scaler = MinMaxScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_test_num_scaled = scaler.transform(X_test_num)

# label_encoder = LabelEncoder()
# X_cat_encoded = label_encoder.fit_transform(X_cat)
# X_test_cat_encoded = label_encoder.transform(X_test_cat)

In [ ]:
pd.get_dummies(X_cat)

In [ ]:
# Split the data into training and validation sets
Xtn_scaled, Xvn_scaled, yt, yv = train_test_split(X_num_scaled, y, test_size=0.25, random_state=888, stratify=y)

# train

In [ ]:
import xgboost as xgb

best_params = {'n_estimators': 2545,
 'learning_rate': 0.030312402618414676,
 'max_depth': 29,
 'lambda': 3.9410705514928686,
 'alpha': 4.837852994429205,
 'subsample': 0.8993275809401318,
 'colsample_bytree': 0.8383987302764818,
 'colsample_bynode': 0.9945482713831311,
 'min_child_weight': 16.36151941228406}

model = xgb.XGBClassifier(**best_params,
                          objective='multi:softmax',
                          device='cpu', # cuda:0 -> method=hist
                          verbosity=0,
                          num_class=2,
                          use_label_encoder=False,
                          seed=888,
                          early_stopping_rounds=10)
model.fit(Xtn_scaled, yt, eval_set=[(Xvn_scaled, yv)], verbose=False)
yv_pred = model.predict(Xvn_scaled)
accuracy = accuracy_score(yv, yv_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(yv, yv_pred))

In [ ]:
import xgboost as xgb

best_params = {'n_estimators': 2545,
 'learning_rate': 0.030312402618414676,
 'max_depth': 29,
 'lambda': 3.9410705514928686,
 'alpha': 4.837852994429205,
 'subsample': 0.8993275809401318,
 'colsample_bytree': 0.8383987302764818,
 'colsample_bynode': 0.9945482713831311,
 'min_child_weight': 16.36151941228406}

stkf = StratifiedKFold(n_splits=9, shuffle=True)  # do not set random_state
model_ls = []
total_acc = 0

for train_idx, val_idx in stkf.split(X_num_scaled, y):
    Xt, Xv = X_num_scaled[train_idx], X_num_scaled[val_idx]
    yt, yv = y[train_idx], y[val_idx]

    model = xgb.XGBClassifier(**best_params,
                          objective='multi:softmax',
                          device='cpu', # cuda:0 -> method=hist
                          verbosity=0,
                          num_class=2,
                          use_label_encoder=False,
                          seed=888,
                          early_stopping_rounds=10)
    model.fit(Xt, yt, eval_set=[(Xv, yv)], verbose=False)
    model_ls.append(model)

    yv_pred = model.predict(Xv)
    accuracy = accuracy_score(yv, yv_pred)
    total_acc += accuracy

# Average accuracy across folds
avg_acc = total_acc / stkf.get_n_splits()

print(f'Validation Accuracy: {avg_acc}')
# print(classification_report(y_val_encoded, y_val_pred))

# predict

## normal

In [ ]:
y_test_pred = model.predict(X_test_num_scaled)
sp_sms.loc[3:, 'Predicted'] = y_test_pred[3:]
sp_sms['Predicted'].value_counts()

In [ ]:
sp_sms['Predicted'] = sp_sms['Predicted'].astype(int)
sp_sms

In [ ]:
# Save the submission to a CSV file
sp_sms.to_csv('datasci_xgb_dropObj.csv', index=False)

## ensemble 9 fold

In [ ]:
pred_cols = []

for i in range(len(model_ls)):
  # Predict the classes
  y_test_pred = model_ls[i].predict(X_test_num_scaled)

  if i == 0:
    pred_df = pd.DataFrame(y_test_pred)
  else:
    pred_df = pd.concat([pred_df, pd.DataFrame(y_test_pred)], axis=1)
  pred_cols.append("fold_" + str(i + 1))
  pred_df.columns = pred_cols
del pred_cols
pred_df

In [ ]:
for i in pred_df.columns:
  print(pred_df[i].value_counts(), "\n")

In [ ]:
ens_pred = pred_df.mode(axis=1)
ens_pred.value_counts()

In [ ]:
sp_sms.loc[3:, 'Predicted'] = np.array(ens_pred[0])[3:]
sp_sms

In [ ]:
sp_sms['Predicted'] = sp_sms['Predicted'].astype(int)
sp_sms

In [ ]:
# Save the submission to a CSV file
sp_sms.to_csv('datasci_xgb9folds_dropObj.csv', index=False)